## 1. 启动类型
> 冷启动
  当启动应用时，后台没有该应用的进程，这时系统会重新创建一个新的进程分配给该应用，这个启动方式就是冷启动。冷启动因为系统会重新创建一个新的进程分配给它，所以会先创建和初始化 Application 类，再创建和初始化 MainActivity 类，最后显示在界面上。

> 热启动
  当启动应用时，后台已有该应用的进程（例：按 back 键、home 键，应用虽然会退出，但是该应用的进程依然会保留在后台，可进入任务列表查看），所以在已有进程的情况下，这种启动会从已有的进程中来启动应用，这个方式叫热启动。热启动因为会从已有的进程中来启动，所以热启动就不会走 Application 这步了，而是直接走 MainActivity。
  
> 首次启动
  首次启动严格来说也是冷启动，之所以把首次启动单独列出来，一般来说，首次启动时间会比非首次启动要久，首次启动会做一些系统初始化工作，如缓存目录的生产，数据库的建立，SharedPreference 的初始化，如果存在多 dex 和插件的情况下，首次启动会有一些特殊需要处理的逻辑，而且对启动速度有很大的影响，所以首次启动的速度非常重要，毕竟影响用户对 App 的第一映像。

---

## 2. 本地启动时间的统计方式
- WaitTime: 返回从 startActivity 到应用第一帧完全显示这段时间。就是总的耗时，包括前一个应用 Activity pause 的时间和新应用启动的时间；
- ThisTime: 表示一连串启动 Activity 的最后一个 Activity 的启动耗时；
- **TotalTime** 表示新应用启动的耗时，包括新进程的启动和 Activity 的启动，但不包括前一个应用 Activity pause 的耗时。
  - 这个时间才是自己应用真正启动的耗时。

```
# 命令
adb shell am start -w packagename/activity

# 输出
$adb shell am start -W com.speed.test/com.speed.test.HomeActivity$
Starting: Intent { act=android.intent.action.MAIN cat=[android.intent.category.LAUNCHER] cmp=com.speed.test/.HomeActivity }
Status: ok
Activity: com.speed.test/.HomeActivity
ThisTime: 496
TotalTime: 496
WaitTime: 503
```

---

## 3. 线上启动时间的统计方式
- 当 App 发到线上之后，基本上都是通过打 Log 的方式将启动时间发送上来。那么在什么位置加启动时间统计的 Log 就尤为重要，Log 添加的位置直接决定启动时间统计的是否准确，同样也会影响启动速度优化效果的判断。


### 3.1 应用的主要启动流程
- 1. 通过 Launcher 启动应用时，点击应用图标后，Launcher 调用 startActivity 启动应用
- 2. Launcher Activity 最终调用 Instrumentation 的 execStartActivity 来启动应用
- 3. Instrumentation 调用 ActivityManagerProxy (ActivityManagerService 在应用进程的一个代理对象) 对象的 startActivity 方法启动 Activity
- 4. 到目前为止所有过程都在 Launcher 进程里面执行，接下来 ActivityManagerProxy 对象跨进程调用 ActivityManagerService (运行在 system_server 进程)的 startActivity 方法启动应用
- 5. ActivityManagerService 的 startActivity 方法经过一系列调用，最后调用 zygoteSendArgsAndGetResult 通过 socket 发送给 zygote 进程，zygote 进程会孵化出新的应用进程
- 6. zygote 进程孵化出新的应用进程后，会执行 ActivityThread 类的 main 方法。在该方法里会先准备好 Looper 和消息队列，然后调用 attach 方法将应用进程绑定到 ActivityManagerService，然后进入 loop 循环，不断地读取消息队列里的消息，并分发消息
- 7. ActivityManagerService 保存应用进程的一个代理对象，然后 ActivityManagerService 通过代理对象通知应用进程创建入口 Activity 的实例，并执行它的生命周期函数

### 3.2 生命周期函数执行流程
- 开发者能做的恰恰是从上述过程中最后一步开始，即 ActivityManagerService 通过代理对象通知应用进程创建入口 Activity 的实例，并执行它的生命周期函数开始，我们的启动时间统计以及启动速度优化也是从这里开始。

```
-> Application 构造函数
-> Application.attachBaseContext()
-> Application.onCreate()
-> Activity 构造函数
-> Activity.setTheme()
-> Activity.onCreate()
-> Activity.onStart
-> Activity.onResume
-> Activity.onAttachedToWindow
-> Activity.onWindowFocusChanged
```

#### 3.2.1 起始时间点
- **冷启动时间**: 一般可以在 Application.attachBaseContext() 开始的位置记录起始时间点(因为在这之前 Context 还没有初始化，一般也干不了什么事情)
- **热启动时间**: 可以在 Activity.onRestart() 中记录起始时间点

#### 3.2.2 结束时间点
- 一个 Activity 走完 onCreate、onStart、onResume 这几个生命周期之后，只是完成了应用自身的一些配置，比如 Activity 主题设置，window 属性的设置，View 树的建立，但是其实后面还需要各个 View 执行 measure、layout、draw 等。所以这个方法中并不准确
- Activity.onWindowFocusChanged: 记录应用启动的结束时间点
  - 在 Activity 焦点发生变化时就会触发，所以要做好判断，去掉不需要的情况。
  - 说明如下

```
/**
 * Called when the current {@link Window} of the activity gains or loses
 * focus.  This is the best indicator of whether this activity is visible
 * to the user. The default implementation clears the key tracking
 * state, so should always be called.
 */
```

---